In [1]:
# Import necessary libraries
import win32file
import win32pipe
import time
import random
import numpy as np

In [2]:
#Generate random array with random values and indices
x = random.randint(1, 20)
y = random.randint(1,20)
print ("x (number of indices) is ", x, "and y (number of values) is ", y)
my_array = np.random.randint(100, size = (x,y))
# print(my_array)

#turns the nested listed into bytes (inconclusive attempt)
# my_message = my_array.tobytes()
# print(my_message)

#it's not the point of the assignment but the pipe can only carry strings for now
my_message = ("I cannot work with lists yet either but here is the string version of a random 2d array"+ str(my_array))

x (number of indices) is  3 and y (number of values) is  12


In [3]:
pipeName="my_pipe"

# Create pipe path
pipe_handle = win32pipe.CreateNamedPipe(
        r'\\.\pipe\\'+pipeName,  # defining pipe name
        win32pipe.PIPE_ACCESS_DUPLEX,   #two way
        win32pipe.PIPE_TYPE_BYTE | win32pipe.PIPE_READMODE_BYTE | win32pipe.PIPE_WAIT, # defining pipe properties
        1, 65536, 65536,
        0,
        None)

# Connect to pipe
win32pipe.ConnectNamedPipe(pipe_handle, None)

# Read from and Write to Pipe
count = 0 
#end communication after 1 exchange, otherwise continues. It should be altered to stop after the pipe is empty...
while True:
    if count == 0:

        # Read message from client
        ret, read_message = win32file.ReadFile(pipe_handle, 1000) 
        print(F'{ret = } Received from c#: ' + read_message.decode('utf-8')) #works for strings
        # print((list(bytes(read_message)))) #turns bytes to integers but doesnt give the original array
        # received = (x.decode('utf-8') for x in read_message) 
        # print(received, "is ", type(received))

        # Write message to client
        ret, length = win32file.WriteFile(pipe_handle, my_message.encode()) 
        #cannot transfer arrays because writefile command is for strings
        print(F'{ret = }, {length = } from WriteFile')

        count = count+1
    else:
   # win32pipe.DisconnectNamedPipe(pipe_handle)
            print("Closing Now...")
            # time.sleep(2)
            win32file.CloseHandle(pipe_handle)
            break

ret = 0 Received from c#: I cannot transfer 2d arrays yet

ret = 0, length = 204 from WriteFile
Closing Now...


Notes:

Parts of the named pipe script including python & c# is adapted from: 

https://www.codeproject.com/Questions/5340484/How-to-send-back-data-through-Python-to-Csharp-thr 


initialise named pipe
pipe3 = win32pipe.CreateNamedPipe(
    r'\\.\pipe\ABC', 
    win32pipe.PIPE_ACCESS_DUPLEX, 
    win32pipe.PIPE_TYPE_MESSAGE, 
    win32pipe.PIPE_READMODE_MESSAGE,
    win32pipe.PIPE_WAIT,
     1, 65536, 65536, 0, None)

handle = CreateNamedPipe(pipeName, openMode , pipeMode , nMaxInstances , nOutBufferSize , nInBufferSize , nDefaultTimeOut , sa )

https://learn.microsoft.com/en-us/windows/win32/api/winbase/nf-winbase-createnamedpipea

pipe_access_dublex = two directional pipe. data can be read and written

pipe_type_message = Data is written to the pipe as a stream of messages. 
    The pipe treats the bytes written during each write operation as a message unit.
    The GetLastError function returns ERROR_MORE_DATA when a message is not read completely. 
    This mode can be used with either PIPE_READMODE_MESSAGE or PIPE_READMODE_BYTE.

pipe_read_mode = Data is read from the pipe as a stream of messages.

pipe_wait = reading and writing continues until the file is drained

pipename should be of the form \\.\pipe\mypipename

connect server to named pipe

win32pipe.ConnectNamedPipe(pipe3, None)

#end connection

win32pipe.DisconnectNamedPipe(pipe3)

win32file.CloseHandle(pipe3)

#read file

resp = win32file.ReadFile(pipe3, 65536)